# ResNet-50 Model Training for Flower Dataset

## Imports

Required imports for training the models

In [ ]:
import cv2
import matplotlib.pyplot as plt
import scipy
import numpy as np
import random
import torch
import torchvision
from tqdm import tqdm
import tarfile
import os
import urllib.request

## Raw dataset downloader functions 

Required functions for the initial dataset download

In [ ]:
def download(url: str, path: str) -> None:
    """
    Function: Download files from a url to a specific directory
    Parameters:
        1. url: File url
        2. path: Target directory
    Returns:
        1. None
    """
    try:
        os.mkdir(path)
    except:
        path = path
    filename = os.path.basename(url)
    urllib.request.urlretrieve(url, f"{path}/{filename}")


def extract(path: str, target: str) -> None:
    """
    Function: Extract tar files from a directory to a specific directory
    Parameters:
        1. path: .tar file directory
        2. target: Target directory
    Returns:
        1. None
    """
    file = tarfile.open(path)
    file.extractall(target)
    file.close()
    os.remove(path)


def dataset(ds_url: str, label_url: str, target: str) -> None:
    """
    Function: Download raw dataset
    Parameters:
        1. ds_url: Raw data url
        2. label_url: Label url
        3. target: Target location for the dataset and label matrix
    Returns:
        1. None
    """
    download(ds_url, target)
    ds_name = os.path.basename(ds_url)
    ds_loc = target + ds_name
    extract(ds_loc, target)
    download(label_url, target)

## Image pre-processing functions

Required functions the process the individual images

In [ ]:
def crop(image: np.ndarray) -> np.ndarray:
    """
    Function: Randomly crop the image into (224, 224) for the model
    Parameters:
        1. image: Image array
    Returns:
        1. Cropped image array
    """
    crop_height, crop_width = 224, 224
    max_x = image.shape[1] - crop_width
    max_y = image.shape[0] - crop_height
    x = np.random.randint(0, max_x)
    y = np.random.randint(0, max_y)
    crop = image[y: y + crop_height, x: x + crop_width]
    return crop


def resize(image: np.ndarray) -> np.ndarray:
    """
    Function: Resize the image into (224, 224) for the model
    Parameters:
        1. image: Image array
    Returns:
        1. Resized image array
    """
    dim = (224, 224)
    resized = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
    return resized


def orient(image: np.ndarray) -> np.ndarray:
    """
    Function: Randomly flip the image along x-axis, y-axis or both
    Parameters:
        1. image: Image array
    Returns:
        1. Flipped image array
    """
    return cv2.flip(image, random.choice([0, 1, -1]))


def section(image: np.ndarray) -> np.ndarray:
    """
    Function: Randomly resize or crop the image array
    Parameters:
        1. image: Image array
    Returns:
        1. Cropped/Resized image array
    """
    functions = {
        1: resize,
        2: crop
    }
    return functions[random.choice([1, 2])](image)


def normalize(matrix: np.ndarray) -> np.ndarray:
    """
    Function: Normalize image array
    Parameters:
        1. matrix: Image array
    Return:
        1. Normalized image array
    """
    return ((matrix / 255.0) - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]

## Dataset pre-processing functions

Required functions to process the raw dataset into acceptable dataset

In [ ]:
def generateDistribution(labels: list[int]) -> dict[int, list[int]]:
    """
    Function: Generate distribution of the data according to its labels
    Parameters:
        1. labels: List of image labels
    Returns:
        1. A dictionary of labels as keys and a list image numbers corresponding that label
    """
    dist = {}
    for l in range(len(labels)):
        if labels[l] not in dist:
            dist[labels[l]] = [l + 1]
        else:
            dist[labels[l]].append(l + 1)
    return dist


def generateDataSet(split: float, dist: dict[int, list[int]]) -> tuple[dict[int, list[int]], dict[int, list[int]]]:
    """
    Function: Split dataset into test dataset and training dataset
    Parameters:
        1. split: Splitting ratio for training data
        2. dist: Dictionary of labels as keys and a list image numbers corresponding that label
    Returns:
        1. Training data distribution dictionary
        2. Testing data distribution dictionary
    """
    trainingDS = {}
    testingDS = {}
    for i in range(len(list(dist.keys()))):
        trainingDS[i + 1] = []
        testingDS[i + 1] = []
    for i in range(len(list(dist.keys()))):
        img_nums = list(dist[i + 1])
        temp = random.sample(img_nums, k=int(len(img_nums) * split))
        trainingDS[i + 1] = temp
        testingDS[i + 1] = [x for x in img_nums if x not in temp]
    return trainingDS, testingDS


def updateDataSet(datasetDict: dict[int, list[int]]) -> list[list[str, int]]:
    """
    Function: Convert data distribution dictionary into a list of a list of image location and its respective label
    Parameters:
        1. datasetDict: Dictionary of labels as keys and a list image numbers corresponding that label
    Returns:
        1. List of a list of image location and its respective label
    """
    dataset = []
    for i in range(len(datasetDict.keys())):
        for j in datasetDict[i + 1]:
            image = './dataset/jpg/image_' + str(j).rjust(5, '0') + '.jpg'
            dataset.append([image, i])
    return dataset


def shuffleDataSet(dataset: list[list[str, int]]) -> list[list[str, int]]:
    """
    Function: Shuffle dataset
    Parameters:
        1. dataset: List of a list of image location and its respective label
    Returns:
        1. Shuffled dataset
    """
    return random.sample(dataset, len(dataset))


def imageTransformation(image: np.ndarray) -> np.ndarray:
    """
    Function: Transform image array randomly
    Parameters:
        1. image: Image array
    Returns:
        1. Randomly flipped, cropped/resized and normalized image array
    """
    return normalize(section(orient(image)))


def updateLabels(labels: np.ndarray, n_classes: int) -> np.ndarray:
    """
    Function: Convert list of labels into a 3D array of size (length of list of labels, total unique number of labels) as per the label probability
    Parameters:
        1. labels: 1D array of labels
        2. n_classes: total unique number of labels
    Returns:
        1. Array of labels
    """
    labels = list(labels)
    labelArr = np.zeros((len(labels), n_classes))
    for i, j in enumerate(labels):
        labelArr[i][j] = 1
    return labelArr

## Utility functions

Required functions for miscellaneous uses

In [ ]:
def accuracy(correctList: list[int], totalList: list[int]) -> tuple[int, int]:
    """
    Function: Calculate the number of correct predictions and total number of predictions
    Parameters:
        1. correctList: List of correct predictions for each label
        2. totalList: List of total predictions for each label
    Returns:
        1. Number of correct predictions
        2. Number of total predictions
    """
    correctList = list(correctList.cpu().numpy())
    totalList = list(totalList.cpu().numpy())
    correct = 0
    total = len(correctList)
    if correctList != totalList:
        for i in range(len(correctList)):
            if correctList[i] == totalList[i]:
                correct += 1
        return correct, total
    else:
        return total, total


def plotAccuracies(correct: list[int], total: list[int], names: list[str], sample: int) -> None:
    """
    Function: Plot a specific number of most accurate and least accurate examples
    Parameters:
        1. correct: List of correct predictions for each label
        2. total: List of total predictions for each label
        3. names: List of names for corresponding labels
        4. sample: Number of samples to be displayed
    Returns:
        1. None
    """
    if sample > len(np.unique([a / b for a, b in zip(correct, total)])) // 2:
        sample = len(np.unique([a / b for a, b in zip(correct, total)])) // 2
        print(
            f'Changing sample size to maximum allowed value. If you want to reduce sample size, please choose between [1, {sample}]')
    yLab = ['Most Accurate', 'Least Accurate']
    # Calculating the percentage of accuracy for each label
    percent = [a / b for a, b in zip(correct, total)]
    # Sorting the list to get the most accurate and least accurate examples
    sorted_list = percent.copy()
    sorted_list.sort()
    sorted_list.reverse()
    disp_names = []
    disp_vals = []
    colors = ('palegreen', 'salmon')
    # Getting the labels and accuracy percentages
    for x in range(len(sorted_list)):
        for y in range(len(percent)):
            if percent[y] == sorted_list[x]:
                if names[y] not in disp_names:
                    if sorted_list[x] not in disp_vals:
                        disp_vals.append(sorted_list[x])
                        disp_names.append(names[y])
    act_vals = disp_vals.copy()
    # Dividing the list into 2 parts; most and least accurate for the given number (sample) of examples
    disp_vals = [disp_vals[:sample], list(np.flip(disp_vals[sample:]))]
    disp_names = [disp_names[:sample], list(np.flip(disp_names[sample:]))]
    # Plotting the pie chart of the respective accuracies
    if sample > 0:
        for i in range(2 * sample):
            row = i // sample
            col = i - (row * sample)
            ax1 = plt.subplot2grid((2, sample), (row, col))
            plt.pie([disp_vals[row][col], 1.0 - disp_vals[row][col]], colors=colors)
            if col == 0:
                plt.ylabel(f'{yLab[row]}')
            plt.text(0, -1.25, f'{round(disp_vals[row][col] * 100, 2)}% Accuracy', ha='center', wrap=True)
            plt.title(disp_names[row][col])
        plt.savefig('./output/test.png')
        plt.show()
    else:
        print(
            f'Changing sample size to minimum allowed value. If you want to increase sample size, please choose sample size between [1,{int(len((act_vals)) / 2)}]')
        plotAccuracies(correct, total, names, 1)

## Model training functions

Main ```train``` and ```test``` functions for model training

In [ ]:
def train(model: torch.nn.Module, optim: torch.optim.Optimizer, loss_fn: torch.nn.Module,
          trainingDS: dict[int, list[int]],
          epochs: int = 300, batch_size: int = 8) -> tuple[list[float], list[float], list[float], list[float]]:
    """
    Function: Train the input model for specific number of epochs (with a specific batch size)
    Parameters:
        1. model: Machine learning model
        2. optim: Model optimizer
        3. loss_fn: Loss function
        4. trainingDS: Training dataset for the model
        5. epochs: Number of epochs
        6. batch_size: Batch size
    Returns:
        1. List of accuracies for the training loop for each epoch
        2. List of accuracies for the validation loop for each epoch
        3. List of losses for the validation loop for each epoch
        4. List of losses for the training loop for each epoch
    """
    trainingAccuracy = []
    validationAccuracy = []
    validationLosses = []
    trainingLosses = []
    minimumAccuracy = 0.5
    for epoch in range(epochs):
        ## Training Loop
        # Splitting the data into training and validation sets
        trainingDS_split, validationDS_split = generateDataSet(0.7 / 0.9, trainingDS)
        validationLosses_value = 0.0
        trainingLosses_value = 0.0
        # Shuffling the training dataset
        shuffledTrainingDS = shuffleDataSet(updateDataSet(trainingDS_split))
        trainingTotal = 0
        trainingCorrect = 0
        validationCorrect = 0
        validationTotal = 0
        if len(shuffledTrainingDS) % batch_size == 0:
            trainEnd = int(len(shuffledTrainingDS) / batch_size)
        else:
            trainEnd = int(len(shuffledTrainingDS) / batch_size) + 1
        for i in tqdm(range(trainEnd)):
            # Dividing the dataset into batches of the given batch size
            trainingBatches = np.asarray(shuffledTrainingDS[i * batch_size:(i + 1) * batch_size]).T[0]
            # Transforming the dataset for accurate training
            trainingBatches = np.asarray(
                [imageTransformation(cv2.imread(x)).astype(np.float32) for x in trainingBatches])
            trainingBatches = torch.tensor(trainingBatches)
            trainingBatches = torch.permute(trainingBatches, (0, 3, 1, 2)).to(device)
            # Extracting the actual labels for the above dataset
            trainingLabels = np.asarray(shuffledTrainingDS[i * batch_size:(i + 1) * batch_size]).T[1].astype(np.int64)
            trainingLabels = updateLabels(trainingLabels, 102)
            trainingLabels = torch.tensor(trainingLabels).to(device)
            trainingLabels_output = model(trainingBatches)
            _, trainingLabels_actual = torch.max(trainingLabels, 1)
            _, trainingLabels_predicted = torch.max(trainingLabels_output, 1)
            # Calculating the training accuracy for current batch
            trainingCorrect_i, trainingTotal_i = accuracy(trainingLabels_actual, trainingLabels_predicted)
            trainingCorrect += trainingCorrect_i
            trainingTotal += trainingTotal_i
            # Calculating the training loss for current batch
            trainingLoss = loss_fn(trainingLabels_output, trainingLabels)
            trainingLosses_value += batch_size * trainingLoss.item()
            optim.zero_grad()
            trainingLoss.backward()
            optim.step()
        print(
            f'Training Accuracy at {epoch + 1} epoch/s: {round((trainingCorrect / trainingTotal) * 100, 2)} % - ({trainingCorrect}/{trainingTotal})')
        # Calculating the training accuracy & training loss for current epoch
        trainingAccuracy.append(round((trainingCorrect / trainingTotal) * 100, 2))
        trainingLosses.append(trainingLosses_value / len(shuffledTrainingDS))
        print(f'Training Loss at {epoch + 1} epoch/s: {trainingLosses_value / (len(shuffledTrainingDS))}')
        ## Validation Loop
        with torch.no_grad():
            # Shuffling the validation dataset
            shuffledValidationDS = shuffleDataSet(updateDataSet(validationDS_split))
            if len(shuffledValidationDS) % batch_size == 0:
                valEnd = int(len(shuffledValidationDS) / batch_size)
            else:
                valEnd = int(len(shuffledValidationDS) / batch_size) + 1
            for j in tqdm(range(valEnd)):
                # Dividing the dataset into batches of the given batch size
                validationBatches = torch.tensor(np.asarray([resize(cv2.imread(x)).astype(np.float32) for x in
                                                             np.asarray(shuffledValidationDS[
                                                                        j * batch_size:(j + 1) * batch_size]).T[0]]))
                validationBatches = torch.permute(validationBatches, (0, 3, 1, 2)).to(device)
                # Extracting the actual labels for the above dataset
                validationLabels = np.asarray(shuffledValidationDS[j * batch_size:(j + 1) * batch_size]).T[1].astype(
                    np.int64)
                validationLabels = updateLabels(validationLabels, 102)
                validationLabels = torch.tensor(validationLabels).to(device)
                validationLabels_output = model(validationBatches)
                # Calculating the validation loss for current batch
                validationLoss = loss_fn(validationLabels_output, validationLabels)
                validationLosses_value += batch_size * validationLoss.item()
                _, validationLabels_actual = torch.max(validationLabels, 1)
                _, validationLabels_predicted = torch.max(validationLabels_output, 1)
                # Calculating the validation accuracy for current batch
                validationCorrect_i, validationTotal_i = accuracy(validationLabels_actual, validationLabels_predicted)
                validationCorrect += validationCorrect_i
                validationTotal += validationTotal_i
            # Saving the most accurate model
            if validationCorrect / validationTotal > minimumAccuracy:
                minimumAccuracy = validationCorrect / validationTotal
                if not os.path.exists('./models/'):
                    os.makedirs('./models/')
                torch.save(model, './models/final.pt')
        # Calculating the validation accuracy & validation loss for current epoch
        validationLosses.append(validationLosses_value / (len(shuffledValidationDS)))
        print(f'Validation Loss at {epoch + 1} epoch/s: {validationLosses_value / (len(shuffledValidationDS))}')
        print(
            f'Validation Accuracy at {epoch + 1} epoch/s: {round((validationCorrect / validationTotal) * 100, 2)} % - ({validationCorrect}/{validationTotal})')
        validationAccuracy.append(round((validationCorrect / validationTotal) * 100, 2))
    # Retraining the model if maximum accuracy is not reached
    if max(validationAccuracy) <= 74.0:
        print('\n\nIncreased training!!!')
        trainingAccuracy_new, validationAccuracy_new, validationLosses_new, trainingLosses_new = train(model, optim,
                                                                                                       loss_fn,
                                                                                                       trainingDS,
                                                                                                       epochs=50)
        trainingAccuracy.extend(trainingAccuracy_new)
        validationAccuracy.extend(validationAccuracy_new)
        validationLosses.extend(validationLosses_new)
        trainingLosses.extend(trainingLosses_new)
    return trainingAccuracy, validationAccuracy, validationLosses, trainingLosses


def test(model: torch.nn.Module, testingDS: dict[int, list[int]], batch_size: int = 8) -> tuple[list[int], list[int]]:
    """
    Function: Test the trained model with a specific batch size
    Parameters:
        1. model: Machine learning model
        2. testingDS: Testing dataset for the model
        3. batch_size: Batch size
    Returns:
        1. List of correct predictions
        2. List of total predictions
    """
    labels = list(range(102))
    correctPredicted = {classVal: 0 for classVal in labels}
    totalPredicted = {classVal: 0 for classVal in labels}
    with torch.no_grad():
        # Shuffling the testing dataset
        shuffledTestingDS = shuffleDataSet(updateDataSet(testingDS))
        if len(shuffledTestingDS) % batch_size == 0:
            testEnd = int(len(shuffledTestingDS) / batch_size)
        else:
            testEnd = int(len(shuffledTestingDS) / batch_size) + 1
        for j in tqdm(range(testEnd)):
            ## Testing Loop
            # Dividing the dataset into batches of the given batch size
            testingBatches = torch.tensor(np.asarray([normalize(resize(cv2.imread(x))).astype(np.float32) for x in
                                                      np.asarray(
                                                          shuffledTestingDS[j * batch_size:(j + 1) * batch_size]).T[
                                                          0]]))
            testingBatches = torch.permute(testingBatches, (0, 3, 1, 2)).to(device)
            # Extracting the actual labels for the above dataset
            testingLabels = np.asarray(shuffledTestingDS[j * batch_size:(j + 1) * batch_size]).T[1].astype(np.int64)
            testingLabels = torch.tensor(testingLabels).to(device)
            testingLabels_output = model(testingBatches)
            _, testingLabels_predicted = torch.max(testingLabels_output, 1)
            # Calculating the accurately predicted labels
            for l, p in zip(testingLabels, testingLabels_predicted):
                if l == p:
                    correctPredicted[labels[l]] += 1
                totalPredicted[labels[l]] += 1
    torch.save(model, './models/final.pt')
    print(
        f'Model Accuracy: {round(100 * (sum(list(correctPredicted.values())) / sum(list(totalPredicted.values()))), 2)}%')
    return list(correctPredicted.values()), list(totalPredicted.values())

## Model inference function

Main function for inference of the trained model

In [ ]:
# Model inference function

def infer(image: np.ndarray, labels: dict[int, str]) -> str:
    """
    Function: Predict the flower name of input image using the trained model
    Parameters:
        1. image: Image array
        2. labels: Dictionary of flower names to their respective labels
    Returns:
        1. Model predicted flower name
    """

    model = torch.load('./models/final.pt')
    model = model.eval().to(device)
    # Pre-processing the input image for the model
    image = (resize(image) / 255.0).astype(np.float32)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = torch.tensor(image)
    image = torch.permute(image, (0, 3, 1, 2)).to(device)
    # Using the model to predict the flower type of input image
    with torch.no_grad():
        out = model(image)
        _, lb_out = torch.max(out, 1)
        lb_out = list(lb_out.cpu().numpy())[0]
    flowerName = labels[lb_out]
    return flowerName, lb_out

## Dataset

Downloads the raw dataset and initialises the flower names

In [ ]:
# Raw dataset downloading

dataset('https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz',
        'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat',
        './dataset/')

# Flower Names:
labels = {
    0: 'Pink Primrose',
    1: 'Hard-Leaved Pocket Orchid',
    2: 'Canterbury Bells',
    3: 'Sweet Pea',
    4: 'English Marigold',
    5: 'Tiger Lily',
    6: 'Moon Orchid',
    7: 'Bird of Paradise',
    8: 'Monkshood',
    9: 'Globe Thistle',
    10: 'Snapdragon',
    11: 'Colts Foot',
    12: 'King Protea',
    13: 'Spear Thistle',
    14: 'Yellow Iris',
    15: 'Globe-Flower',
    16: 'Purple Coneflower',
    17: 'Peruvian Lily',
    18: 'Balloon Flower',
    19: 'Giant White Arum Lily',
    20: 'Fire Lily',
    21: 'Pincushion Flower',
    22: 'Fritillary',
    23: 'Red Ginger',
    24: 'Grape Hyacinth',
    25: 'Corn Poppy',
    26: 'Prince of Wales Feathers',
    27: 'Stemless Gentian',
    28: 'Artichoke',
    29: 'Sweet William',
    30: 'Carnation',
    31: 'Garden Phlox',
    32: 'Love in the Mist',
    33: 'Mexican Aster',
    34: 'Sea Holly',
    35: 'Ruby-Lipped Cattleya',
    36: 'Cape Flower',
    37: 'Masterwort',
    38: 'Siam Tulip',
    39: 'Lenten Rose',
    40: 'Barbeton Daisy',
    41: 'Daffodil',
    42: 'Sword Lily',
    43: 'Poinsettia',
    44: 'Bolero Deep Blue',
    45: 'Wallflower',
    46: 'Marigold',
    47: 'Buttercup',
    48: 'Oxeye Daisy',
    49: 'Common Dandelion',
    50: 'Petunia',
    51: 'Wild Pansy',
    52: 'Primula',
    53: 'Sunflower',
    54: 'Pelargonium',
    55: 'Bishop of Llandaff',
    56: 'Gaura',
    57: 'Geranium',
    58: 'Orange Dahlia',
    59: 'Pink-Yellow Dahlia',
    60: 'Cautleya Spicata',
    61: 'Japanese Anemone',
    62: 'Black-Eyed Susan',
    63: 'Silverbush',
    64: 'Californian Poppy',
    65: 'Osteospermum',
    66: 'Spring Crocus',
    67: 'Bearded Iris',
    68: 'Windflower',
    69: 'Tree Poppy',
    70: 'Gazania',
    71: 'Azalea',
    72: 'Water Lily',
    73: 'Rose',
    74: 'Thorn Apple',
    75: 'Morning Glory',
    76: 'Passion Flower',
    77: 'Lotus',
    78: 'Toad Lily',
    79: 'Anthurium',
    80: 'Frangipani',
    81: 'Clematis',
    82: 'Hibiscus',
    83: 'Columbine',
    84: 'Desert-Rose',
    85: 'Tree Mallow',
    86: 'Magnolia',
    87: 'Cyclamen',
    88: 'Watercress',
    89: 'Canna Lily',
    90: 'Hippeastrum',
    91: 'Bee Balm',
    92: 'Ball Moss',
    93: 'Foxglove',
    94: 'Bougainvillea',
    95: 'Camellia',
    96: 'Mallow',
    97: 'Mexican Petunia',
    98: 'Bromelia',
    99: 'Blanket Flower',
    100: 'Trumpet Creeper',
    101: 'Blackberry Lily'
}

In [ ]:
# Data Distribution

y = [len(generateDistribution(scipy.io.loadmat('./dataset/imagelabels.mat')['labels'][0])[x]) for x in generateDistribution(scipy.io.loadmat('./dataset/imagelabels.mat')['labels'][0]).keys()]
plt.bar(list(range(len(y))), y)
plt.savefig('./output/dist.png')
plt.show()

## Model creation

Initialisation of the model with additional layer for our custom dataset and initialisation of model optimizers and loss function

In [ ]:
# Importing model and setting the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torchvision.models.resnet50(weights=torchvision.models.resnet.ResNet50_Weights.IMAGENET1K_V2)

# Modifying the model to accomodate our dataset
new_model = torch.nn.Sequential(
    model,
    torch.nn.Linear(1000, 786),
    torch.nn.ReLU(inplace=True),
    torch.nn.Linear(786, 256),
    torch.nn.ReLU(inplace=True),
    torch.nn.Linear(256, 102),
    torch.nn.Softmax(dim=1)
)
model = new_model.to(device)

# Importing the model optimizer and the loss functions
optim = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=0.0001)
loss_fn = torch.nn.CrossEntropyLoss()

## Training

Main training of the model

In [ ]:
rawLabels = generateDistribution(scipy.io.loadmat('./dataset/imagelabels.mat')['labels'][0])
# Training the model
trainingDS_split, testingDS_split = generateDataSet(0.9, rawLabels)
trainingAccuracy, validationAccuracy, validationLoss, trainingLoss = train(model, optim, loss_fn, trainingDS_split)

## Training plots

Plotting the accuracies and losses of the trained model

In [ ]:
# Plotting the losses over each epoch
plt.plot(validationLoss)
plt.plot(trainingLoss)
plt.legend(['Validation Loss', 'Training Loss'], loc='upper right')
plt.savefig('./output/loss.png')
plt.show()

In [ ]:
# Plotting the accuracies over each epoch
plt.plot(validationAccuracy)
plt.plot(trainingAccuracy)
plt.legend(['Validation Accuracy', 'Training Accuracy'], loc='lower right')
plt.savefig('./output/acc.png')
plt.show()

## Testing

Main testing of the trained model

In [ ]:
# Testing the model
corr, total = test(model, testingDS_split)

## Testing plots

Accuracy plots of the tested model

In [ ]:
# Plotting the most and least accurate flower types
names = list(labels.values())
plotAccuracies(corr, total, names, 1)

## Inference

Inference of the trained model for specific input

In [ ]:
imageID = random.randint(0, 8188)
image = cv2.imread('./dataset/jpg/image_' + str(imageID + 1).rjust(5, '0') + '.jpg').astype(np.float32)
# Generating the overlay based on the accuracy of the prediction
overlay_image = np.zeros(image.shape).astype(np.float32)
actImage_label = scipy.io.loadmat('./dataset/imagelabels.mat')['labels'][0][imageID] - 1
predFlowerName, predImage_label = infer(image, labels)
actFlowerName = labels[actImage_label]
channel = 2
symbol = '✗'
if actImage_label == predImage_label:
    channel = 1
    symbol = '✓'
overlay_image[:, :, channel] = 255.0
image = cv2.addWeighted(image, 0.55, overlay_image, 0.45, 0).astype(np.uint8)
plt.imshow(image[:, :, ::-1], cmap='Greens')
plt.title(
    f'Actual flower type: {actFlowerName}\nPredicted flower type: {predFlowerName} {symbol}')
plt.savefig('./output/inf.png')
plt.show()